In [1]:
include("HighDimMixedModels.jl")

Main.HighDimMM

In [2]:
using LinearAlgebra
using DataFrames
using StableRNGs; rng = StableRNG(1);
using StatsModels
using MixedModels
import .HighDimMM   #https://docs.julialang.org/en/v1/manual/variables-and-scoping/

**Set up data and formula, specify factor variables(CategoricalTerm)**

In [3]:
#df = DataFrame(y = rand(rng, 9), a = 1:9, b = rand(rng, 9), c = repeat(["d","e","f"], 3), d = vcat(repeat([1,2], 4),1))
df = DataFrame(y = rand(rng, 9), a = 1:9, b = rand(rng, 9), c = repeat(["d","e","f"], 3), d = b = rand(rng, 9))
f = @formula(y ~  a + b + c + d)
contrasts = Dict( :a => ContinuousTerm, :b=>ContinuousTerm, :c => CategoricalTerm)

Dict{Symbol, UnionAll} with 3 entries:
  :a => ContinuousTerm
  :b => ContinuousTerm
  :c => CategoricalTerm

In [4]:
df

,y,a,b,c,d
,Float64,Int64,Float64,String,Float64
1,0.585195,1,0.236782,d,0.752223
2,0.0773379,2,0.943741,e,0.314815
3,0.716628,3,0.445671,f,0.858522
4,0.320357,4,0.763679,d,0.698713
5,0.653093,5,0.145071,e,0.757746
6,0.236639,6,0.021124,f,0.419294
7,0.709684,7,0.152545,d,0.412607
8,0.557787,8,0.617492,e,0.454589
9,0.05079,9,0.481531,f,0.380933


In [5]:
f

FormulaTerm
Response:
  y(unknown)
Predictors:
  a(unknown)
  b(unknown)
  c(unknown)
  d(unknown)

**try for f2**

In [ ]:
# try for f2
f2 = @formula(y ~ 1 + a + b + (1|c))

In [ ]:
sch = MixedModels.schema(f2,df,contrasts)

In [ ]:
form = MixedModels.apply_schema(f2, sch, LinearMixedModel)

In [ ]:
form = MixedModels.apply_schema(f2, sch, HighDimMM.highDimMixedModel)

**back here**

In [6]:
sch = MixedModels.schema(f,df,contrasts)

StatsModels.Schema with 5 entries:
  c => c
  b => b
  d => d
  a => a
  y => y


In [7]:
form.rhs.terms[[2,3]]

LoadError: UndefVarError: form not defined

In [8]:
form = apply_schema(f, sch)

FormulaTerm
Response:
  y(continuous)
Predictors:
  a(continuous)
  b(continuous)
  c(DummyCoding:3→2)
  d(continuous)

In [ ]:
typeof(form)

In [ ]:
typeof(form.rhs.terms[1])

In [ ]:
modelmatrix(form.rhs.terms[1],df)

In [ ]:
Vector(1:length(form.rhs.terms))

In [ ]:
[1,2]

In [ ]:
df[:,["b","c"]]

In [19]:
y,pred = modelcols(form, df);

In [ ]:
pred = modelmatrix(form, df);

In [ ]:
pred

In [ ]:
typeof(pred)

### Number of variables indicates M,X,Z

In [13]:
HMM = HighDimMM.highDimMixedModel(f, df, contrasts, 1, 1)

┌ Warning: n >= p in high dimensional matrix
└ @ Main.HighDimMM /Users/zyxu/Documents/julia/highDimMM/bricks.jl:12


Main.HighDimMM.highDimMixedModel{Float64}(y ~ a + b + c + d, Main.HighDimMM.highDimMat{Float64, Matrix{Float64}}([0.236781883208121; 0.9437409715735081; … ; 0.617492416565387; 0.48153065407402607]), Main.HighDimMM.XMat{Float64, Matrix{Float64}}([1.0 0.0; 2.0 1.0; … ; 8.0 1.0; 9.0 0.0]), [0.0 0.7522231521093703; 0.0 0.31481548090434086; … ; 0.0 0.45458919106592477; 1.0 0.38093255230858825], [0.5851946422124186, 0.07733793456911231, 0.7166282400543453, 0.3203570514066232, 0.6530930076222579, 0.2366391513734556, 0.7096838914472361, 0.5577872440804086, 0.05079002172175784])

In [14]:
# Three matrix
HMM.M.M; HMM.X.X; HMM.Z.Z

9×2 Matrix{Float64}:
 0.0  0.752223
 0.0  0.314815
 1.0  0.858522
 0.0  0.698713
 0.0  0.757746
 1.0  0.419294
 0.0  0.412607
 0.0  0.454589
 1.0  0.380933

### indices indicates M,X,Z

In [9]:
HMM = HighDimMM.highDimMixedModel(f, df, contrasts, 1, [2,4],3)

LoadError: MethodError: no method matching modelmatrix(::ContinuousTerm{Float64})
[0mClosest candidates are:
[0m  modelmatrix(::Union{Tuple{Union{Tuple{AbstractTerm, Vararg{AbstractTerm, N} where N}, AbstractTerm}, Vararg{Union{Tuple{AbstractTerm, Vararg{AbstractTerm, N} where N}, AbstractTerm}, N} where N}, AbstractTerm}, [91m::Any[39m; hints, mod) at /Users/zyxu/.julia/packages/StatsModels/JZLpf/src/modelframe.jl:109
[0m  modelmatrix([91m::StatsModels.TableRegressionModel[39m, [91m::Any...[39m; kwargs...) at /Users/zyxu/.julia/packages/StatsModels/JZLpf/src/statsmodel.jl:28
[0m  modelmatrix([91m::ModelFrame[39m; data) at /Users/zyxu/.julia/packages/StatsModels/JZLpf/src/modelframe.jl:147
[0m  ...

In [ ]:
HMM.M.M

In [ ]:
HMM.X.X

In [ ]:
HMM.Z.Z

In [ ]:
HMM.M * HMM.X

In [ ]:
HMM.M.M * HMM.X.X

In [ ]:
# response
HMM.y

In [33]:
size(HMM.X,2)

2

$$
\mathbf{y} \text { is } N_{n}(\mathbf{X} \boldsymbol{\beta}, \mathbf{\Sigma}), \quad \text { where } \quad \mathbf{\Sigma}= \sigma_{z}^{2}
\mathbf{Z} \mathbf{Z}^{\prime}+\sigma^{2} \mathbf{I}_{n} 
$$

Let $$\mathbf{P}=\mathbf{I}-\mathbf{H}=\mathbf{I}-\mathbf{X}\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-} \mathbf{X}^{\prime}$$

$$\mathbf{K}=\mathbf{C}\mathbf{P} \in R^{(n-r)\times n}$$

$$
Ky \sim N_{n-r}\left(\mathbf{0}, \mathbf{K} \mathbf{\Sigma} \mathbf{K}^{\prime}\right) = N_{n-r}\left[\mathbf{0}, \mathbf{K}\left(\sigma_{z}^{2} \mathbf{Z} \mathbf{Z}^{\prime}+\sigma^{2} \mathbf{I}_{n} \right) \mathbf{K}^{\prime}\right]
$$

To simplify, we assume fixed effect as full rank, where r is the number of fixed effects.

In [15]:
A = hcat(HMM.M.M, HMM.X.X)
P = I - A*inv(transpose(A)*A)*transpose(A)
u,s,v = svd(P)
C = transpose(u[:,1:6]) # simplify: assume fixed effect full rank
K = C*P

6×9 Matrix{Float64}:
  6.39454e-18  -0.307199    …  -0.174553    0.537463    -0.555444
 -0.979033      0.0837545       0.0042392  -0.00141918   0.0609113
 -7.41899e-19   0.33468        -0.0465556  -0.167309    -0.0137589
  6.09135e-18  -0.368244       -0.32402     0.27034      0.312723
  2.79907e-17   0.149667        0.0968698   0.476231     0.205796
 -1.28753e-18  -0.00554263  …  -0.772305   -0.0194777    0.392502

The objective function is:
$$
\frac{n-r}{2} \ln (2 \pi)-\frac{1}{2} \ln \left|\mathbf{K} \Sigma \mathbf{K}^{\prime}\right|-\frac{1}{2} \mathbf{y}^{\prime} \mathbf{K}^{\prime}\left(\mathbf{K} \Sigma \mathbf{K}^{\prime}\right)^{-1} \mathbf{K} \mathbf{y}
$$

In [11]:
using NLopt

In [48]:
function negLogLik(sigma::Vector, K::AbstractMatrix, Z::AbstractMatrix, y::Vector)
    n = length(y) 
    Sigma = sigma[1]*Z*transpose(Z) + sigma[2]*diagm(ones(n))
    negLog = -1/2*log(det(K*Sigma*K)) - 1/2*transpose(y)*transopse(K)*inv(K*Sigma*K)*K*y
    return negLog
end

negLogLik (generic function with 2 methods)

In [54]:
HMM.Z.Z::AbstractMatrix

9×2 Matrix{Float64}:
 0.0  0.752223
 0.0  0.314815
 1.0  0.858522
 0.0  0.698713
 0.0  0.757746
 1.0  0.419294
 0.0  0.412607
 0.0  0.454589
 1.0  0.380933

In [49]:
negLogLik(sigma = [1,1], K = K, Z = HMM.Z.Z, y = HMM.y)

LoadError: MethodError: no method matching negLogLik(; sigma=[1, 1], K=[6.3945370816102936e-18 -0.307198967569933 … 0.5374631772388195 -0.5554440004491068; -0.9790326601851597 0.08375454647121926 … -0.0014191831068713625 0.06091133045140232; … ; 2.799065050142728e-17 0.14966747227114716 … 0.4762313964947574 0.20579620403486606; -1.2875342187767984e-18 -0.005542627056688555 … -0.019477697691443828 0.3925023964554191], Z=[0.0 0.7522231521093703; 0.0 0.31481548090434086; … ; 0.0 0.45458919106592477; 1.0 0.38093255230858825], y=[0.5851946422124186, 0.07733793456911231, 0.7166282400543453, 0.3203570514066232, 0.6530930076222579, 0.2366391513734556, 0.7096838914472361, 0.5577872440804086, 0.05079002172175784])
[0mClosest candidates are:
[0m  negLogLik([91m::Vector{T} where T[39m, [91m::Vector{T} where T[39m, [91m::AbstractMatrix{T} where T[39m, [91m::AbstractMatrix{T} where T[39m, [91m::Vector{T} where T[39m) at In[23]:1[91m got unsupported keyword arguments "sigma", "K", "Z", "

In [24]:
opt = Opt(:LN_COBYLA, 2)
opt.min_objective = negLogLik
opt.xtol_rel = 1e-5

1.0e-5

In [29]:
(optf,optx,ret) = optimize(opt, [1,1])

(4.232700236833981e246, [1.0, 1.0], :FORCED_STOP)

In [ ]:
using NLopt

function myfunc(x::Vector, grad::Vector)
    if length(grad) > 0
        grad[1] = 0
        grad[2] = 0.5/sqrt(x[2])
    end
    return sqrt(x[2])
end

function myconstraint(x::Vector, grad::Vector, a, b)
    if length(grad) > 0
        grad[1] = 3a * (a*x[1] + b)^2
        grad[2] = -1
    end
    (a*x[1] + b)^3 - x[2]
end

opt = Opt(:LD_MMA, 2)
opt.lower_bounds = [-Inf, 0.]
opt.xtol_rel = 1e-4

opt.min_objective = myfunc
inequality_constraint!(opt, (x,g) -> myconstraint(x,g,2,0), 1e-8)
inequality_constraint!(opt, (x,g) -> myconstraint(x,g,-1,1), 1e-8)

(minf,minx,ret) = optimize(opt, [1.234, 5.678])
numevals = opt.numevals # the number of function evaluations
println("got $minf at $minx after $numevals iterations (returned $ret)")

In [45]:
using DataFrames
a = NA

LoadError: UndefVarError: NA not defined